In [7]:
# !pip install flask_wtf

# from flask import Flask, render_template, redirect, url_for, session, flash
# from flask_sqlalchemy import SQLAlchemy
# from flask_wtf import FlaskForm
# from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
# from wtforms.validators import InputRequired, Email, Length, ValidationError
# from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
# from flask_bcrypt import Bcrypt

In [10]:
# !pip freeze > requirement.txt

In [9]:
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
import pandas as pd

app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///user.db'
app.config['SECRET_KEY'] = '123456'
    
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    plan = db.relationship('Plan', backref='writer', lazy='dynamic')


class Plan(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    plan_body = db.Column(db.String(100))
    plan_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)


class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")


class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")

# 新增note
class NewPlanForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Title"})
    # 我把lengthmax改成由100到500
    plan_body = StringField("Plan Body", validators=[InputRequired(), Length(max=500)], render_kw={"placeholder":  "Plan Body"})
    submit = SubmitField("Add Plan")

@app.route("/")
def index():
    return render_template('index.html')


@app.route("/game")
def games():
    # This csv file is obtained from Assignmrnt_4.
    df = pd.read_csv("game.csv")
    list = df.to_dict('records')
    return render_template('game.html', entries = list)

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("view_plan"))
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)

@app.route("/register", methods=["GET", "POST"])
# sqlalchemy
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)

@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))


@app.route('/new-plan', methods=['GET','POST'])
@login_required
def new_plan():
    form = NewPlanForm()
    if form.validate_on_submit():
        new_plan = Plan(title=form.title.data, plan_body=form.plan_body.data, writer=current_user)
        db.session.add(new_plan)
        db.session.commit()
        return redirect(url_for('view_plan'))
    return render_template('new_plan.html', title='New Plan', form=form)


@app.route('/my-plan', methods=['GET','POST'])
@login_required
def view_plan():
    plan = Plan.query.filter_by(writer=current_user).all()
    return render_template('my_plan.html', plan=plan, title='My Plan')


@app.route('/delete-plan/<int:plan_id>', methods=['GET',"POST"])
def delete_plan(plan_id):
    plan = Plan.query.get_or_404(plan_id)
    db.session.delete(plan)
    db.session.commit()
    return redirect(url_for('view_plan'))


if __name__ == '__main__':
    app.run('localhost', 0000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\ProgramData\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:53125/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Oct/2021 21:13:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 21:13:55] "GET /static/css/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 21:13:55] "GET /static/assets/img/ipad.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 21:13:55] "GET /static/assets/img/bg_pic1.png HTTP/1.1" 404 -
127.0.0.1 - - [23/Oct/2021 21:13:56] "GET /static/assets/img/demo-image-01.jpg HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 21:13:56] "GET /static/assets/img/demo-image-02.jpg HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 21:13:56] "GET /static/assets/img/bg-masthead.jpg HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 21:13:56] "GET /sta